# GPU Check

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-256b2290-36af-03cf-5a92-66ced509a847)


# Git Clone

In [2]:
!git clone https://github.com/jackyjsy/CVPR21Chal-SLR.git

Cloning into 'CVPR21Chal-SLR'...
remote: Enumerating objects: 673, done.
remote: Counting objects: 100% (673/673), done.
remote: Compressing objects: 100% (531/531), done.
remote: Total 673 (delta 216), reused 585 (delta 130), pack-reused 0
Receiving objects: 100% (673/673), 51.71 MiB | 24.85 MiB/s, done.
Resolving deltas: 100% (216/216), done.


# Drive Mount

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessed data copying

In [5]:
import shutil
import os

In [6]:
data_type = 'joint'
#data_type = 'joint_motion'
#data_type = 'bone'
#data_type = 'bone_motion'

# Train Data

In [7]:
original = f'/content/drive/.shortcut-targets-by-id/1MwfX64WpeVyGAW0MhcasmbihkfgJwbtY/Darkpose_outputs/train_val/train_val_data_{data_type}.npy'
target = f'/content/data/sign/27_2/'
os.makedirs(os.path.dirname(target), exist_ok=True)
shutil.copy(original, target)

original = f'/content/drive/.shortcut-targets-by-id/1MwfX64WpeVyGAW0MhcasmbihkfgJwbtY/Darkpose_outputs/train_val/train_val_labels.pkl'
target = f'/content/data/sign/27_2/'
os.makedirs(os.path.dirname(target), exist_ok=True)
shutil.copy(original, target)

'/content/data/sign/27_2/train_val_labels.pkl'

# Valid Data

In [8]:
original = f'/content/drive/.shortcut-targets-by-id/1MwfX64WpeVyGAW0MhcasmbihkfgJwbtY/Darkpose_outputs/test_data_{data_type}.npy'
target = r'/content/data/sign/27_2/'
os.makedirs(os.path.dirname(target), exist_ok=True)
shutil.copy(original, target)

original = r'/content/drive/.shortcut-targets-by-id/1MwfX64WpeVyGAW0MhcasmbihkfgJwbtY/Darkpose_outputs/test_label.pkl'
target = r'/content/data/sign/27_2/'
os.makedirs(os.path.dirname(target), exist_ok=True)
shutil.copy(original, target)

'/content/data/sign/27_2/test_label.pkl'

# Imports

In [9]:
from __future__ import print_function
import argparse
from argparse import ArgumentParser,Namespace
import os
import time
import numpy as np
import yaml
import pickle
from collections import OrderedDict
# torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import shutil
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR
import random
import inspect
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

# Seed fixing

In [10]:
def init_seed(_):
    torch.cuda.manual_seed_all(1)
    torch.manual_seed(1)
    np.random.seed(1)
    random.seed(1)
    # torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# To import the data or model in load_data and load_model

In [11]:
def import_class(name):
    components = name.split('.')
    mod = __import__(components[0])  # import return model
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod

# Main Class

Ami only line 247 nije nije disi

to create the saving directory

In [12]:
class Processor():
    """ 
        Processor for Skeleton-based Action Recgnition
    """

    def __init__(self, arg):

        arg.model_saved_name = f"/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/finetune/{data_type}/save_models/"+ arg.Experiment_name
        arg.work_dir = f"/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/finetune/{data_type}/work_dir/" + arg.Experiment_name
        self.arg = arg
        self.save_arg()
        if arg.phase == 'train':
            if not arg.train_feeder_args['debug']:
                if os.path.isdir(arg.model_saved_name):
                    print('log_dir: ', arg.model_saved_name, 'already exist')
                    answer = input('delete it? y/n:')
                    if answer == 'y':
                        shutil.rmtree(arg.model_saved_name)
                        print('Dir removed: ', arg.model_saved_name)
                        input(
                            'Refresh the website of tensorboard by pressing any keys')
                    else:
                        print('Dir not removed: ', arg.model_saved_name)

        self.global_step = 0
        self.load_model()
        self.load_optimizer()
        self.load_data()
        self.lr = self.arg.base_lr
        self.best_acc = 0

    def load_data(self):
        Feeder = import_class(self.arg.feeder)
        self.data_loader = dict()
        if self.arg.phase == 'train':
            self.data_loader['train'] = torch.utils.data.DataLoader(
                dataset=Feeder(**self.arg.train_feeder_args),
                batch_size=self.arg.batch_size,
                shuffle=True,
                num_workers=self.arg.num_worker,
                drop_last=True,
                worker_init_fn=init_seed)
        self.data_loader['test'] = torch.utils.data.DataLoader(
            dataset=Feeder(**self.arg.test_feeder_args),
            batch_size=self.arg.test_batch_size,
            shuffle=False,
            num_workers=self.arg.num_worker,
            drop_last=False,
            worker_init_fn=init_seed)

    def load_model(self):
        output_device = self.arg.device[0] if type(
            self.arg.device) is list else self.arg.device
        self.output_device = output_device
        Model = import_class(self.arg.model)
        shutil.copy2(inspect.getfile(Model), self.arg.work_dir)
        self.model = Model(**self.arg.model_args).cuda(output_device)
        # print(self.model)
        self.loss = nn.CrossEntropyLoss().cuda(output_device)
        # self.loss = LabelSmoothingCrossEntropy().cuda(output_device)

        if self.arg.weights:
            self.print_log('Load weights from {}.'.format(self.arg.weights))
            if '.pkl' in self.arg.weights:
                with open(self.arg.weights, 'r') as f:
                    weights = pickle.load(f)
            else:
                weights = torch.load(self.arg.weights)

            weights = OrderedDict(
                [[k.split('module.')[-1],
                  v.cuda(output_device)] for k, v in weights.items()])

            for w in self.arg.ignore_weights:
                if weights.pop(w, None) is not None:
                    self.print_log('Sucessfully Remove Weights: {}.'.format(w))
                else:
                    self.print_log('Can Not Remove Weights: {}.'.format(w))

            try:
                self.model.load_state_dict(weights)
            except:
                state = self.model.state_dict()
                diff = list(set(state.keys()).difference(set(weights.keys())))
                print('Can not find these weights:')
                for d in diff:
                    print('  ' + d)
                state.update(weights)
                self.model.load_state_dict(state)

        if type(self.arg.device) is list:
            if len(self.arg.device) > 1:
                self.model = nn.DataParallel(
                    self.model,
                    device_ids=self.arg.device,
                    output_device=output_device)

    def load_optimizer(self):
        if self.arg.optimizer == 'SGD':

            params_dict = dict(self.model.named_parameters())
            params = []

            for key, value in params_dict.items():
                decay_mult = 0.0 if 'bias' in key else 1.0

                lr_mult = 1.0
                weight_decay = 1e-4

                params += [{'params': value, 'lr': self.arg.base_lr, 'lr_mult': lr_mult,
                            'decay_mult': decay_mult, 'weight_decay': weight_decay}]

            self.optimizer = optim.SGD(
                params,
                momentum=0.9,
                nesterov=self.arg.nesterov)
        elif self.arg.optimizer == 'Adam':
            self.optimizer = optim.Adam(
                self.model.parameters(),
                lr=self.arg.base_lr,
                weight_decay=self.arg.weight_decay)
        else:
            raise ValueError()

        self.lr_scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1,
                                              patience=10, verbose=True,
                                              threshold=1e-4, threshold_mode='rel',
                                              cooldown=0)

    def save_arg(self):
        # save arg
        arg_dict = vars(self.arg)

        if not os.path.exists(self.arg.work_dir):
            os.makedirs(self.arg.work_dir)
            os.makedirs(self.arg.work_dir + '/eval_results')

        with open('{}/config.yaml'.format(self.arg.work_dir), 'w') as f:
            yaml.dump(arg_dict, f)

    def adjust_learning_rate(self, epoch):
        if self.arg.optimizer == 'SGD' or self.arg.optimizer == 'Adam':
            if epoch < self.arg.warm_up_epoch:
                lr = self.arg.base_lr * (epoch + 1) / self.arg.warm_up_epoch
            else:
                lr = self.arg.base_lr * (
                    0.1 ** np.sum(epoch >= np.array(self.arg.step)))
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = lr
            return lr
        else:
            raise ValueError()

    def print_time(self):
        localtime = time.asctime(time.localtime(time.time()))
        self.print_log("Local current time :  " + localtime)

    def print_log(self, str, print_time=True):
        if print_time:
            localtime = time.asctime(time.localtime(time.time()))
            str = "[ " + localtime + ' ] ' + str
        print(str)
        if self.arg.print_log:
            with open('{}/log.txt'.format(self.arg.work_dir), 'a') as f:
                print(str, file=f)

    def record_time(self):
        self.cur_time = time.time()
        return self.cur_time

    def split_time(self):
        split_time = time.time() - self.cur_time
        self.record_time()
        return split_time

    def train(self, epoch, save_model=False):
        self.model.train()
        self.print_log('Training epoch: {}'.format(epoch + 1))
        loader = self.data_loader['train']
        self.adjust_learning_rate(epoch)
        loss_value = []
        self.record_time()
        timer = dict(dataloader=0.001, model=0.001, statistics=0.001)
        process = tqdm(loader)
        if epoch >= self.arg.only_train_epoch:
            print('only train part, require grad')
            for key, value in self.model.named_parameters():
                if 'DecoupleA' in key:
                    value.requires_grad = True
                    print(key + '-require grad')
        else:
            print('only train part, do not require grad')
            for key, value in self.model.named_parameters():
                if 'DecoupleA' in key:
                    value.requires_grad = False
                    print(key + '-not require grad')
        for batch_idx, (data, label, index) in enumerate(process):
            self.global_step += 1
            # get data
            data = Variable(data.float().cuda(
                self.output_device), requires_grad=False)
            label = Variable(label.long().cuda(
                self.output_device), requires_grad=False)
            timer['dataloader'] += self.split_time()

            # forward
            if epoch < 100:
                keep_prob = -(1 - self.arg.keep_rate) / 100 * epoch + 1.0
            else:
                keep_prob = self.arg.keep_rate
            output = self.model(data, keep_prob)

            if isinstance(output, tuple):
                output, l1 = output
                l1 = l1.mean()
            else:
                l1 = 0
            loss = self.loss(output, label) + l1

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_value.append(loss.data)
            timer['model'] += self.split_time()

            value, predict_label = torch.max(output.data, 1)
            acc = torch.mean((predict_label == label.data).float())

            self.lr = self.optimizer.param_groups[0]['lr']

            if self.global_step % self.arg.log_interval == 0:
                self.print_log(
                    '\tBatch({}/{}) done. Loss: {:.4f}  lr:{:.6f}'.format(
                        batch_idx, len(loader), loss.data, self.lr))
            timer['statistics'] += self.split_time()

        # statistics of time consumption and loss
        proportion = {
            k: '{:02d}%'.format(int(round(v * 100 / sum(timer.values()))))
            for k, v in timer.items()
        }

        state_dict = self.model.state_dict()
        weights = OrderedDict([[k.split('module.')[-1],
                                v.cpu()] for k, v in state_dict.items()])

        os.makedirs(os.path.dirname(f'/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/finetune/{data_type}/save_models/'), exist_ok=True)
        torch.save(weights, self.arg.model_saved_name +
                   '-' + str(epoch) + '.pt')

    def eval(self, epoch, save_score=False, loader_name=['test'], wrong_file=None, result_file=None):
        if wrong_file is not None:
            f_w = open(wrong_file, 'w')
        if result_file is not None:
            f_r = open(result_file, 'w')
        self.model.eval()
        with torch.no_grad():
            self.print_log('Eval epoch: {}'.format(epoch + 1))
            for ln in loader_name:
                loss_value = []
                score_frag = []
                right_num_total = 0
                total_num = 0
                loss_total = 0
                step = 0
                process = tqdm(self.data_loader[ln])

                for batch_idx, (data, label, index) in enumerate(process):
                    data = Variable(
                        data.float().cuda(self.output_device),
                        requires_grad=False)
                    label = Variable(
                        label.long().cuda(self.output_device),
                        requires_grad=False)

                    with torch.no_grad():
                        output = self.model(data)

                    if isinstance(output, tuple):
                        output, l1 = output
                        l1 = l1.mean()
                    else:
                        l1 = 0
                    loss = self.loss(output, label)
                    score_frag.append(output.data.cpu().numpy())
                    loss_value.append(loss.data.cpu().numpy())

                    _, predict_label = torch.max(output.data, 1)
                    step += 1

                    if wrong_file is not None or result_file is not None:
                        predict = list(predict_label.cpu().numpy())
                        true = list(label.data.cpu().numpy())
                        for i, x in enumerate(predict):
                            if result_file is not None:
                                f_r.write(str(x) + ',' + str(true[i]) + '\n')
                            if x != true[i] and wrong_file is not None:
                                f_w.write(str(index[i]) + ',' +
                                        str(x) + ',' + str(true[i]) + '\n')
                score = np.concatenate(score_frag)

                if 'UCLA' in arg.Experiment_name:
                    self.data_loader[ln].dataset.sample_name = np.arange(
                        len(score))

                accuracy = self.data_loader[ln].dataset.top_k(score, 1)
                if accuracy > self.best_acc:
                    self.best_acc = accuracy
                    score_dict = dict(
                        zip(self.data_loader[ln].dataset.sample_name, score))

                    with open(f'/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/finetune/{data_type}/work_dir/' + arg.Experiment_name + '/eval_results/best_acc' + '.pkl'.format(
                            epoch, accuracy), 'wb') as f:
                        pickle.dump(score_dict, f)

                print('Eval Accuracy: ', accuracy,
                    ' model: ', self.arg.model_saved_name)

                score_dict = dict(
                    zip(self.data_loader[ln].dataset.sample_name, score))
                self.print_log('\tMean {} loss of {} batches: {}.'.format(
                    ln, len(self.data_loader[ln]), np.mean(loss_value)))
                for k in self.arg.show_topk:
                    self.print_log('\tTop{}: {:.2f}%'.format(
                        k, 100 * self.data_loader[ln].dataset.top_k(score, k)))

                with open(f'/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/finetune/{data_type}/work_dir/' + arg.Experiment_name + '/eval_results/epoch_' + str(epoch) + '_' + str(accuracy) + '.pkl'.format(
                        epoch, accuracy), 'wb') as f:
                    pickle.dump(score_dict, f)
        return np.mean(loss_value)
    def start(self):
        if self.arg.phase == 'train':
            self.print_log('Parameters:\n{}\n'.format(str(vars(self.arg))))
            self.global_step = self.arg.start_epoch * \
                len(self.data_loader['train']) / self.arg.batch_size
            for epoch in range(self.arg.start_epoch, self.arg.num_epoch):
                save_model = ((epoch + 1) % self.arg.save_interval == 0) or (
                    epoch + 1 == self.arg.num_epoch)

                self.train(epoch, save_model=save_model)

                val_loss = self.eval(
                    epoch,
                    save_score=self.arg.save_score,
                    loader_name=['test'])

                # self.lr_scheduler.step(val_loss)

            print('best accuracy: ', self.best_acc,
                  ' model_name: ', self.arg.model_saved_name)

        elif self.arg.phase == 'test':
            if not self.arg.test_feeder_args['debug']:
                wf = self.arg.model_saved_name + '_wrong.txt'
                rf = self.arg.model_saved_name + '_right.txt'
            else:
                wf = rf = None
            if self.arg.weights is None:
                raise ValueError('Please appoint --weights.')
            self.arg.print_log = False
            self.print_log('Model:   {}.'.format(self.arg.model))
            self.print_log('Weights: {}.'.format(self.arg.weights))
            self.eval(epoch=self.arg.start_epoch, save_score=self.arg.save_score,
                      loader_name=['test'], wrong_file=wf, result_file=rf)
            self.print_log('Done.\n')

# Passed Arguments

In [16]:
arg = Namespace(
    Experiment_name=f'{data_type}_27_2_finetune', 
    base_lr=0.01, 
    batch_size=64, 
    config=f'/content/CVPR21Chal-SLR/SL-GCN/config/sign/finetune/train_{data_type}.yaml', 
    device=[0], 
    eval_interval=5, 
    feeder='feeders.feeder.Feeder', 
    groups=8, 
    ignore_weights=[], 
    keep_rate=0.9, 
    log_interval=100, 
    model='model.decouple_gcn_attn.Model', 
    model_args={
        'num_class': 226, 
        'num_point': 27, 
        'num_person': 1, 
        'graph': 'graph.sign_27.Graph', 
        'groups': 16, 
        'block_size': 41, 
        'graph_args': {'labeling_mode': 'spatial'}
    }, 
    model_saved_name='', 
    nesterov=True, 
    num_epoch=100, 
    num_worker=32, 
    only_train_epoch=1, 
    only_train_part=True, 
    optimizer='SGD', 
    phase='train', 
    print_log=True, 
    save_interval=2, 
    save_score=False, 
    seed=1, 
    show_topk=[1, 5], 
    start_epoch=0, ###############################################################
    step=[50], 
    test_batch_size=64, 
    test_feeder_args={
        'data_path': f'/content/data/sign/27_2/test_data_{data_type}.npy', 
        'label_path': f'/content/data/sign/27_2/test_label.pkl', 
        'random_mirror': False, 
        'normalization': True
    }, 
    train_feeder_args={
        'data_path': f'/content/data/sign/27_2/train_val_data_{data_type}.npy', 
        'label_path': f'/content/data/sign/27_2/train_val_labels.pkl', 
        'debug': False, 
        'random_choose': True, 
        'window_size': 100, 
        'random_shift': True, 
        'normalization': True, 
        'random_mirror': True, 
        'random_mirror_p': 0.5, 
        'is_vector': False
    }, 
    warm_up_epoch=0, 
    weight_decay=0.0001, 
    ##############################################
    # To continue use specific path else None    #
    ##############################################
    weights=f'/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/{data_type}/save_models/sign_{data_type}_final-196.pt', 
    work_dir='./work_dir/temp'
)

# Main cell to run
Personal suggestion:

Ei cell ta ekta copy koire run koris as amader onek baar e train korte hobe and re run korle prev output erase hoye jay.

Also koy step por por latest .pt file ta nije nije download koire nis just in case.

In [ ]:
%cd /content/CVPR21Chal-SLR/SL-GCN
init_seed(0)
processor = Processor(arg)
processor.start()
%cd /content

/content/CVPR21Chal-SLR/SL-GCN
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
[ Sat Aug 14 15:09:33 2021 ] Load weights from /content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/joint/save_models/sign_joint_final-196.pt.


/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:29: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(conv.weight, mode='fan_out')
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:30: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(conv.bias, 0)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:34: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(bn.weight, scale)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:35: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(bn.bias, 0)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:113: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.Linear_bias, 1e-6)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:119: UserWarning: To copy 

32560


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


3742
[ Sat Aug 14 15:09:44 2021 ] Parameters:
{'Experiment_name': 'joint_27_2_finetune', 'base_lr': 0.01, 'batch_size': 64, 'config': '/content/CVPR21Chal-SLR/SL-GCN/config/sign/finetune/train_joint.yaml', 'device': [0], 'eval_interval': 5, 'feeder': 'feeders.feeder.Feeder', 'groups': 8, 'ignore_weights': [], 'keep_rate': 0.9, 'log_interval': 100, 'model': 'model.decouple_gcn_attn.Model', 'model_args': {'num_class': 226, 'num_point': 27, 'num_person': 1, 'graph': 'graph.sign_27.Graph', 'groups': 16, 'block_size': 41, 'graph_args': {'labeling_mode': 'spatial'}}, 'model_saved_name': '/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/Final_train/finetune/joint/save_models/joint_27_2_finetune', 'nesterov': True, 'num_epoch': 100, 'num_worker': 32, 'only_train_epoch': 1, 'only_train_part': True, 'optimizer': 'SGD', 'phase': 'train', 'print_log': True, 'save_interval': 2, 'save_score': False, 'seed': 1, 'show_topk': [1, 5], 'start_epoch': 0, 'step': [50], 'test_batch_size': 64, 'test_feeder_a

...

  0%|          | 0/508 [00:00<?, ?it/s]

only train part, require grad
l1.gcn1.DecoupleA-require grad
l2.gcn1.DecoupleA-require grad
l3.gcn1.DecoupleA-require grad
l4.gcn1.DecoupleA-require grad
l5.gcn1.DecoupleA-require grad
l6.gcn1.DecoupleA-require grad
l7.gcn1.DecoupleA-require grad
l8.gcn1.DecoupleA-require grad
l9.gcn1.DecoupleA-require grad
l10.gcn1.DecoupleA-require grad


  9%|▊         | 44/508 [00:42<07:05,  1.09it/s]

[ Sat Aug 14 19:35:52 2021 ] 	Batch(43/508) done. Loss: 0.0012  lr:0.010000


 26%|██▌       | 130/508 [02:00<05:45,  1.09it/s]

In [ ]:
%cd /content/CVPR21Chal-SLR/SL-GCN
init_seed(0)
processor = Processor(arg)
processor.start()
%cd /content

/content/CVPR21Chal-SLR/SL-GCN


/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:29: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(conv.weight, mode='fan_out')
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:30: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(conv.bias, 0)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:34: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(bn.weight, scale)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:35: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(bn.bias, 0)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:113: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.Linear_bias, 1e-6)
/content/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:119: UserWarning: To copy 

Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
[ Thu Aug  5 13:28:24 2021 ] Load weights from /content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/train2/idk/sign_joint_final-30.pt.
28142


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/439 [00:00<?, ?it/s]

28142
[ Thu Aug  5 13:28:31 2021 ] Parameters:
{'Experiment_name': 'sign_joint_final', 'base_lr': 0.1, 'batch_size': 64, 'config': '/content/CVPR21Chal-SLR/SL-GCN/config/sign/train/train_joint.yaml', 'device': [0], 'eval_interval': 5, 'feeder': 'feeders.feeder.Feeder', 'groups': 8, 'ignore_weights': [], 'keep_rate': 0.9, 'log_interval': 100, 'model': 'model.decouple_gcn_attn.Model', 'model_args': {'num_class': 226, 'num_point': 27, 'num_person': 1, 'graph': 'graph.sign_27.Graph', 'groups': 16, 'block_size': 41, 'graph_args': {'labeling_mode': 'spatial'}}, 'model_saved_name': './save_models/sign_joint_final', 'nesterov': True, 'num_epoch': 250, 'num_worker': 32, 'only_train_epoch': 1, 'only_train_part': True, 'optimizer': 'SGD', 'phase': 'train', 'print_log': True, 'save_interval': 2, 'save_score': False, 'seed': 1, 'show_topk': [1, 5], 'start_epoch': 0, 'step': [150, 200], 'test_batch_size': 64, 'test_feeder_args': {'data_path': '/content/data/sign/27_2/train_data_joint.npy', 'label_pa

...

[ Thu Aug  5 15:14:19 2021 ] 	Batch(343/439) done. Loss: 0.0079  lr:0.025000


  0%|          | 0/440 [00:00<?, ?it/s]

[ Thu Aug  5 15:17:47 2021 ] Eval epoch: 5


 45%|████▌     | 199/440 [03:30<04:13,  1.05s/it]

KeyboardInterrupt: ignored

# To copy all the output files in /content/CVPR21Chal-SLR/SL-GCN/save_models

In [ ]:
import os, shutil
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [ ]:
copytree('/content/CVPR21Chal-SLR/SL-GCN/save_models','/content/drive/MyDrive/Colab Notebooks/CVPR21-Bh/train2/irdk')

# personal debugging

In [ ]:
parser = get_parser()
if p.config is not None:
    with open(p.config, 'r') as f:
        default_arg = yaml.load(f)
    key = vars(p).keys()
    for k in default_arg.keys():
        if k not in key:
            print('WRONG ARG: {}'.format(k))
            assert (k in key)
    parser.set_defaults(**default_arg)

NameError: ignored

In [ ]:
p = Namespace(
    Experiment_name='', 
    base_lr=0.01, 
    batch_size=256, 
    config='/content/CVPR21Chal-SLR/SL-GCN/config/sign/train/train_joint.yaml', 
    device=0, 
    eval_interval=5, 
    feeder='feeder.feeder', 
    groups=8, 
    ignore_weights=[], 
    keep_rate=0.9, 
    log_interval=100, 
    model=None, 
    model_args={}, 
    model_saved_name='', 
    nesterov=False, 
    num_epoch=80, 
    num_worker=32, 
    only_train_epoch=0, 
    only_train_part=True, 
    optimizer='SGD', 
    phase='train', 
    print_log=True, 
    save_interval=2, 
    save_score=False, 
    seed=1, 
    show_topk=[1, 5], 
    start_epoch=0, 
    step=[20, 40, 60], 
    test_batch_size=256, 
    test_feeder_args={}, 
    train_feeder_args={}, 
    warm_up_epoch=0, 
    weight_decay=0.0005, 
    weights=None, 
    work_dir='./work_dir/temp')


In [ ]:
Namespace(
    Experiment_name='sign_joint_final', 
    base_lr=0.1, 
    batch_size=64, 
    config='/content/CVPR21Chal-SLR/SL-GCN/config/sign/train/train_joint.yaml', 
    device=[0, 1, 2, 3], 
    eval_interval=5, 
    feeder='feeders.feeder.Feeder', 
    groups=8, 
    ignore_weights=[], 
    keep_rate=0.9, 
    log_interval=100, 
    model='model.decouple_gcn_attn.Model', 
    model_args={
        'num_class': 226, 
        'num_point': 27, 
        'num_person': 1, 
        'graph': 'graph.sign_27.Graph', 
        'groups': 16, 
        'block_size': 41, 
        'graph_args': {'labeling_mode': 'spatial'}
    }, 
    model_saved_name='', 
    nesterov=True, 
    num_epoch=250, 
    num_worker=32, 
    only_train_epoch=1, 
    only_train_part=True, 
    optimizer='SGD', 
    phase='train', 
    print_log=True, 
    save_interval=2, 
    save_score=False, 
    seed=1, 
    show_topk=[1, 5], 
    start_epoch=0, 
    step=[150, 200], 
    test_batch_size=64, 
    test_feeder_args={
        'data_path': './data/sign/27_2/val_data_joint.npy', 
        'label_path': './data/sign/27_2/val_gt.pkl', 
        'random_mirror': False, 
        'normalization': True
    }, 
    train_feeder_args={
        'data_path': './data/sign/27_2/train_data_joint.npy', 
        'label_path': './data/sign/27_2/train_label.pkl', 
        'debug': False, 
        'random_choose': True, 
        'window_size': 100, 
        'random_shift': True, 
        'normalization': True, 
        'random_mirror': True, 
        'random_mirror_p': 0.5, 
        'is_vector': False
    }, 
    warm_up_epoch=20, 
    weight_decay=0.0001, 
    weights=None, 
    work_dir='./work_dir/temp'
)


In [ ]:
!pwd

/content/CVPR21Chal-SLR/SL-GCN


In [ ]:
%cd /content/data/sign/27_2
!ls

[Errno 20] Not a directory: '/content/data/sign/27_2'
/content/drive/.shortcut-targets-by-id/1GhEIIcgxzMyFdfLihYOKuui9LUxm_E7T/copy_test_123
test_data_bone_motion.npy   test_label.pkl		 train_data_joint.npy
test_data_bone.npy	    train_data_bone_motion.npy	 train_label.pkl
test_data_joint_motion.npy  train_data_bone.npy
test_data_joint.npy	    train_data_joint_motion.npy
